In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

filepath = 'F:/contest/kuaishou/'
app_path = 'app_launch_log.csv'
activity_path = 'user_activity_log.csv'
video_path = 'video_create_log.csv'
register_path = 'user_register_log.csv'

data_reg = pd.read_csv(filepath + register_path)
data_app = pd.read_csv(filepath + app_path)
data_acti = pd.read_csv(filepath + activity_path)
data_video = pd.read_csv(filepath + video_path)

In [4]:
num_train = 3
train_reg = []
train_app = []
train_acti = []
train_video = []
test = []
dayofpredict = 17
for i in range(num_train):

    train_reg.append(data_reg[data_reg.date <= (16+i*7)])
    train_app.append(data_app[(data_app.date <= (16+i*7)) & (data_app.date >= (1+i*7))])
    train_acti.append(data_acti[(data_acti.date <= (16+i*7)) & (data_acti.date >= (1+i*7))])
    train_video.append(data_video[(data_video.date <= (16+i*7)) & (data_video.date >= (1+i*7))])
for i in range(num_train - 1):
    test.append(data_app['id'][(data_app.date>= dayofpredict+i*7) & (data_app.date <= dayofpredict + i*7 + 6)])

In [134]:
test = []
for i in range(num_train - 1):
    test.append(data_app['id'][(data_app.date>= dayofpredict+i*7) & (data_app.date <= dayofpredict + i*7 + 6)])


In [116]:
#注册日志特征




#启动日志特征
def count_num(data):
    return pd.DataFrame(data.groupby(['id']).size())

def gap_last(data, predict_day):
    tmp = data.groupby(['id'])
    gap = []
    last = []
    maxcontinue = []
    for i in tmp:
        gap.append(i[1].date.diff().mean())
        last.append(predict_day - max(i[1].date))
    return pd.DataFrame({'id':sorted(set(data['id'])),0:gap}),pd.DataFrame({'id':sorted(set(data['id'])),0:last})

    
# 行为日志特征
def count_page(data):
    tmp = data.groupby(['id'])
    page_0 = []
    page_1 = []
    page_2 = []
    page_3 = []
    page_4 = []
    for i in tmp:
        page_0.append((i[1].page == 0).sum())
        page_1.append((i[1].page == 1).sum())
        page_2.append((i[1].page == 2).sum())
        page_3.append((i[1].page == 3).sum())
        page_4.append((i[1].page == 4).sum())
    return (pd.DataFrame({'id':sorted(set(data['id'])),0:page_0}),pd.DataFrame({'id':sorted(set(data['id'])),0:page_1}),
            pd.DataFrame({'id':sorted(set(data['id'])),0:page_2}),pd.DataFrame({'id':sorted(set(data['id'])),0:page_3}),pd.DataFrame({'id':sorted(set(data['id'])),0:page_4}))
def count_action(data):
    tmp = data.groupby(['id'])
    action_0 = []
    action_1 = []
    action_2 = []
    action_3 = []
    action_4 = []
    action_5 = []
    for i in tmp:
        action_0.append((i[1].action == 0).sum())
        action_1.append((i[1].action == 1).sum())
        action_2.append((i[1].action == 2).sum())
        action_3.append((i[1].action == 3).sum())
        action_4.append((i[1].action == 4).sum())
        action_5.append((i[1].action == 5).sum())
    return (pd.DataFrame({'id':sorted(set(data['id'])),0:action_0}),pd.DataFrame({'id':sorted(set(data['id'])),0:action_1}),
            pd.DataFrame({'id':sorted(set(data['id'])),0:action_2}),pd.DataFrame({'id':sorted(set(data['id'])),0:action_3}),
            pd.DataFrame({'id':sorted(set(data['id'])),0:action_4}),pd.DataFrame({'id':sorted(set(data['id'])),0:action_5}))
def maxnum_sameauthor(data):
    tmp = data.groupby(['id'])
    sameauthor = []
    samevideo = []
    for i in tmp:
        sameauthor.append(max(i[1].author.value_counts()))
        samevideo.append(max(i[1].video.value_counts()))
    return pd.DataFrame({'id':sorted(set(data['id'])),0:sameauthor}),pd.DataFrame({'id':sorted(set(data['id'])),0:samevideo})

#拍摄日志特征


In [133]:
%%time
feature = []
for i in range(num_train):
    #注册日志
    dayofpredict = 17 + i*7
    feature.append(train_reg[i])
    feature[i]['date'] = dayofpredict - feature[i]['date']
    feature[i] = pd.concat([feature[i], pd.get_dummies(feature[i].source, prefix = 'source')], axis = 1).drop(['source'], axis = 1)
    print('注册日志特征完成-------')
    #启动日志
    feature[i] = pd.merge(feature[i],count_num(train_app[i]).rename({0:'launch_num'}, axis = 1).reset_index(),on = 'id', how = 'left')
    tmp1,tmp2 = gap_last(train_app[i],dayofpredict)
    feature[i] = pd.merge(feature[i],tmp1.rename({0:'launch_gap'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp2.rename({0:'launch_last'}, axis = 1), on = 'id', how = 'left')
    print('启动日志特征完成-------')
   
    #行为日志
    feature[i] = pd.merge(feature[i],count_num(train_acti[i]).rename({0:'acti_num'}, axis = 1).reset_index(), on = 'id', how = 'left')
#     tmp1 = feature[i]['acti_num']/feature[i]['launch_num']
#     feature[i] = pd.merge(feature[i],tmp1.rename({0:'acti_meandays'}, axis = 1).reset_index(), on = 'id', how = 'left')
    feature[i]['acti_meandays'] = feature[i]['acti_num']/feature[i]['launch_num']
    tmp1,tmp2 = gap_last(train_acti[i],dayofpredict)
    feature[i] = pd.merge(feature[i],tmp1.rename({0:'acti_gap'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp2.rename({0:'acti_last'}, axis = 1), on = 'id', how = 'left')
    tmp1,tmp2,tmp3,tmp4,tmp5 = count_page(train_acti[i])
    feature[i] = pd.merge(feature[i],tmp1.rename({0:'page_0'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp2.rename({0:'page_1'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp3.rename({0:'page_2'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp4.rename({0:'page_3'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp5.rename({0:'page_4'}, axis = 1), on = 'id', how = 'left')
    for j in range(5):
        
        feature[i]['page_'+str(j)+'_freq'] = feature[i]['page_'+str(i)]/feature[i]['acti_num']
    tmp1,tmp2,tmp3,tmp4,tmp5,tmp6 = count_action(train_acti[i])
    feature[i] = pd.merge(feature[i],tmp1.rename({0:'action_0'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp2.rename({0:'action_1'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp3.rename({0:'action_2'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp4.rename({0:'action_3'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp5.rename({0:'action_4'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp6.rename({0:'action_5'}, axis = 1), on = 'id', how = 'left')
    for j in range(6):
        feature[i]['action_'+str(j)+'_freq'] = feature[i]['action_'+str(j)]/feature[i]['acti_num'] 
    tmp1,tmp2 = maxnum_sameauthor(train_acti[i])
    feature[i] = pd.merge(feature[i],tmp1.rename({0:'sameauthor'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp2.rename({0:'samevideo'}, axis = 1), on = 'id', how = 'left')
    print('行为日志特征完成-------')
    
    #拍摄日志
    feature[i] = pd.merge(feature[i],count_num(train_video[i]).rename({0:'video_num'}, axis = 1).reset_index(), on = 'id', how = 'left')
    tmp1,tmp2 = gap_last(train_video[i],dayofpredict)
    feature[i] = pd.merge(feature[i],tmp1.rename({0:'video_gap'}, axis = 1), on = 'id', how = 'left')
    feature[i] = pd.merge(feature[i],tmp2.rename({0:'video_last'}, axis = 1), on = 'id', how = 'left')
    

F:\python\venv\anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


注册日志特征完成-------
启动日志特征完成-------
行为日志特征完成-------
注册日志特征完成-------
启动日志特征完成-------
行为日志特征完成-------
注册日志特征完成-------
启动日志特征完成-------
行为日志特征完成-------
Wall time: 12min 21s


In [145]:
#提取标签
def get_label0(data):
    if data.id in test[0]:
        return 1
    else:
        return 0
def get_label1(data):
    if data.id in test[1]:
        return 1
    else:
        return 0
feature[0]['label'] = feature[0].apply(get_label0, axis = 1)
feature[1]['label'] = feature[1].apply(get_label1, axis = 1)

In [298]:
import lightgbm as lgb


clf = lgb.LGBMClassifier(max_depth = 10,unbalanced = True, n_estimators = 1000)
clf.fit(data_train[usefulcol].as_matrix(), data_train.label,
        eval_set = (feature[1][usefulcol].as_matrix(), 
        feature[1]['label']),
        verbose = 100 )

[100]	valid_0's binary_logloss: 0.188339
[200]	valid_0's binary_logloss: 0.168649
[300]	valid_0's binary_logloss: 0.151678
[400]	valid_0's binary_logloss: 0.13841
[500]	valid_0's binary_logloss: 0.12769
[600]	valid_0's binary_logloss: 0.117391
[700]	valid_0's binary_logloss: 0.10887
[800]	valid_0's binary_logloss: 0.101493
[900]	valid_0's binary_logloss: 0.0953648
[1000]	valid_0's binary_logloss: 0.0895311


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=10, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1, unbalanced=True)

In [171]:
from sklearn.metrics import f1_score

def score(real,pred):
    #dataframe
    F1 = f1_score(real,pred)
    return 'self', F1, True
#     print("Precision=",Precision,"| Recall=",Recall)
#     print("F1=",F1)

In [297]:
data_train 

,id,date,device,source_0,source_1,source_2,source_3,source_4,source_5,source_6,...,samevideo,video_num,video_gap,video_last,label,action_1_freq,action_2_freq,action_3_freq,action_4_freq,action_5_freq
0,16,13,2010,1,0,0,0,0,0,0,...,3.0,0.0,0.000000,0.0,0,0.017544,0.017544,0.000000,0.0,0.0
1,98,16,28,1,0,0,0,0,0,0,...,1.0,1.0,0.000000,1.0,0,0.000000,0.000000,0.000000,0.0,0.0
2,105,12,5,0,0,1,0,0,0,0,...,8.0,0.0,0.000000,0.0,0,0.006803,0.003401,0.170068,0.0,0.0
3,218,11,1144,0,1,0,0,0,0,0,...,5.0,0.0,0.000000,0.0,0,0.005831,0.017493,0.000000,0.0,0.0
4,225,7,582,0,1,0,0,0,0,0,...,2.0,0.0,0.000000,0.0,1,0.000000,0.033333,0.000000,0.0,0.0
5,336,1,188,0,0,1,0,0,0,0,...,3.0,0.0,0.000000,0.0,1,0.000000,0.000000,0.000000,0.0,0.0
6,422,5,13,0,0,0,0,1,0,0,...,0.0,0.0,0.000000,0.0,0,0.000000,0.000000,0.000000,0.0,0.0
7,607,1,8,1,0,0,0,0,0,0,...,1.0,0.0,0.000000,0.0,1,0.000000,0.000000,0.000000,0.0,0.0
8,711,6,16,0,1,0,0,0,0,0,...,2.0,0.0,0.000000,0.0,0,0.000000,0.200000,0.000000,0.0,0.0
9,746,14,593,1,0,0,0,0,0,0,...,1.0,0.0,0.000000,0.0,1,0.000000,0.000000,0.000000,0.0,0.0


In [314]:
id.to_csv(filepath+'baseline.csv', index = False)

In [301]:
res = clf.predict_proba(feature[2][usefulcol].as_matrix())[:,1]
# res = clf.predict(feature[2][usefulcol].as_matrix())